<a href="https://colab.research.google.com/github/DEP04929/ESICMDatathon2026/blob/main/ExtractingData_20260114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites for Amsterdam UMC DB


In [ ]:
# sets *your* project id
PROJECT_ID = "esicmdatathon2026" #@param {type:"string"}


In [ ]:
# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [ ]:
import os
from google.colab import auth

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')


In [ ]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 50000


In [ ]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config


In [ ]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns
sns.set_style('darkgrid')

# all possible ventilator patients
this will be used as criteria of person ids for all subsequent extractions

In [ ]:
%%bigquery ventpatlist --project $PROJECT_ID
select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'

# ? Diagnosis: surgical or non surgical
All entries in condition occurrence are admission diagnosis. They are uncoded, so free text search is necessary.


In [ ]:
%%bigquery surg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
), surg as (
select person_id, condition_start_datetime, condition_source_value
, case when lower(condition_source_value) like '%non%surg%' then 'non-surgical' else 'surgical' end as condition_type
from condition_occurrence
where condition_source_value like '%surg%' --matches surgical and non surgical
  and person_id in (select person_id from vlist)
)
select * from surg
PIVOT ( count(*) for condition_type in ('surgical', 'non-surgical'))
order by person_id, condition_start_datetime

In [ ]:
surg.head(20)

In [ ]:
top_10_conditions = surg.groupby('condition_source_value')['person_id'].nunique().nlargest(10)
print(top_10_conditions)

# Heart rate

In [ ]:
%%bigquery hf_ekg --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
select person_id, measurement_datetime, value_as_number as hr_ekg
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in (
	21490872 --Heart rate.beat-to-beat by EKG
)
order by person_id, measurement_datetime

This shows the full range of all heart rate possibilities

In [ ]:
hf_ekg.head(50)


# Arterial blood pressure

In [ ]:
%%bigquery bp --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, bp as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 21490853 then 'ABPs'
     when measurement_concept_id = 21490851 then 'ABPd'
     when measurement_concept_id = 21490852 then 'ABPm'
--     when measurement_concept_id = 21492241 then 'NiBPm'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 21490851 -- ABP d
  , 21490852 -- ABP m
  , 21490853 -- ABPs
--  , 21492241 -- NiBPm
)
)
select * from bp
PIVOT ( max(value_as_number) for measure_type in ('ABPs', 'ABPd', 'ABPm')) --, 'NiBPm'
order by person_id, measurement_datetime

In [ ]:
bp

# Breath rate or respiratory rate

Breath rate from ventilator and respiratory rate from monitor gets mixed up over the years of data recording.

In [ ]:
%%bigquery rr --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, rr as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 3043148 then 'BRvent'
     when measurement_concept_id = 3026892 then 'BRspontv'
     when measurement_concept_id = 1175625 then 'BRspont'
     when measurement_concept_id = 3024171 then 'RR'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 3043148 --Breath rate mechanical
                                 , 3026892 -- Breath rate spontaneous --on ventilator
                                 , 1175625 -- Breath rate spontaneous
                                 , 3024171 -- Respiratory rate
                                 )
)
select * from rr
PIVOT ( max(value_as_number) for measure_type in ('BRvent', 'BRspontv','BRspont', 'RR'))
order by person_id, measurement_datetime

In [ ]:
# rr[rr['BRvent']>90] # plausibility check
# rr[rr['person_id']==23444] # testing purpose, random person

# rr[rr['BRspont']>90] # plausibility check
rr[rr['person_id']==39444] # testing purpose, random person


# BGA

In [ ]:
%%bigquery bga --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
), br as ( select person_id, measurement_datetime
  , case when measurement_concept_id = 3010421 then 'pH'
     when measurement_concept_id = 3027315 then 'PaO2'
		 when measurement_concept_id = 3013290 then 'PaCO2'
		 when measurement_concept_id = 3006576 then 'HCO3'
		 when measurement_concept_id = 3012501 then 'BE'
		 when measurement_concept_id = 3047181 then 'Lactate'
     when measurement_concept_id = 3024928 then 'O2_venous'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
         3010421 -- pH of Blood
											,3027315 -- Oxygen [Partial pressure] in Blood
											,3013290 -- Carbon dioxide [Partial pressure] in Blood
											,3006576 -- Bicarbonate [Moles/volume] in Blood
											,3012501 -- Base excess in Blood by calculation
											,3047181 -- Lactate [Moles/volume] in Blood
                      ,3024928 -- Oxygen saturation in Venous blood
  )
  and value_as_number is not null
  and person_id in (select person_id from vlist)
  )
  select * from br
  PIVOT (max(value_as_number) for concept_name in ('Lactate', 'pH', 'PaO2', 'PaCO2', 'HCO3','BE','O2_venous'))
  order by person_id, measurement_datetime

In [ ]:
bga

# PEEP

In [ ]:
%%bigquery peep --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
 vent as
( select person_id, measurement_datetime
  , case when measurement_concept_id = 3012410 then 'TV'
       when measurement_concept_id = 2000000222 then 'MVspont'
       when measurement_concept_id = 3022875 then 'PEEP'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
        3012410 -- Tidal volume setting Ventilator
        , 3022875 -- Positive end expiratory pressure setting Ventilator
        , 2000000222 -- Minute volume spontaneous
  )
  and value_as_number is not null
  and person_id in (select person_id from vlist)
)
select * from vent
  PIVOT (max(value_as_number) for concept_name in ('MVspont','TV', 'PEEP'))
order by person_id, measurement_datetime

In [ ]:
peep


# FiO2

FiO2 exists for NIV too.

In [ ]:
%%bigquery FiO2 --project esicmdatathon2026
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
fio2 as (
select person_id, measurement_datetime
, case when measurement_concept_id in ( 3025408, 2000000204) then 'FiO2'
       when measurement_concept_id = 2000000203 then 'FiO2niv'
       end as concept_name
, value_as_number
from measurement m
where measurement_concept_id in (2000000203 -- FiO2 NIV
  , 3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
  --, 2000000204 -- fio2 setting
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from fio2
PIVOT (max(value_as_number) for concept_name in ('FiO2', 'FiO2niv'))
order by person_id, measurement_datetime

In [ ]:
FiO2

# Labs: creatinine, bilirubin, hemoglobin and co.


In [ ]:
%%bigquery lab --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
c as (
select person_id, measurement_datetime
, case when measurement_concept_id = 3020564 then 'Cr'
       when measurement_concept_id = 3005772 then 'Br'
       when measurement_concept_id = 40762351 then 'Hb'
       when measurement_concept_id = 3007461 then 'Plt'
       when measurement_concept_id = 3010813 then 'wbc'
       when measurement_concept_id = 3000285 then 'Na'
       end as concept_name
, value_as_number
from measurement m
where measurement_concept_id in (3020564 -- Creatinine [Moles/volume] in Serum or Plasma
  , 3005772 -- Bilirubin.conjugated [Moles/volume] in Serum or Plasma
  , 40762351 -- Hemoglobin [Moles/volume] in Blood'
  , 3007461 -- Platelets [#/volume] in Blood
  , 3010813 -- Leukocytes [#/volume] in Blood
  , 3000285 -- Sodium [Moles/volume] in Blood
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from c
PIVOT (max(value_as_number) for concept_name in ('Cr', 'Br', 'Hb', 'Plt', 'wbc', 'Na'))
order by person_id, measurement_datetime


In [ ]:
lab.describe()

# CAM score?

In [ ]:
%%bigquery test --project esicmdatathon2026
select *
from observation
where observation_concept_id in (3042082 --Confusion Assessment Method (CAM)
)
limit 100

In [ ]:
test

In [ ]:
%%bigquery test --project esicmdatathon2026
select person_id, measurement_date, stddev_samp(value_as_number) as sd
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id in ( 21490872 --Heart rate.beat-to-beat by EKG
) and value_as_number > 0 and value_as_number < 300
group by person_id, measurement_date

In [ ]:
%%bigquery test --project $PROJECT_ID
select c.concept_name, m.measurement_source_value
, max(measurement_date) as last_date_used
, count(distinct person_id) as person_count
, count(*) as number_of_entries
from measurement m
inner join concept c
on m.measurement_concept_id = c.concept_id
where measurement_concept_id = 3024171
group by concept_name, measurement_source_value
order by count(*) desc



In [ ]:
test

# Drugs? Vasopressors?

Quantity is there, but dose is not there, for drug exposure. I am unsure mg/ 24 hours unit meaning?  

In [ ]:
%%bigquery vasopressor --project esicmdatathon2026
select person_id, drug_exposure_start_datetime, drug_exposure_end_datetime, drug_concept_id, quantity
from drug_exposure
where drug_concept_id in (19006871 -- argipressin
                      )
  limit 100

In [ ]:
test